# Basic, Part B: Motor and Move

From *APS Python Training for Bluesky Data Acquisition*.

**Objective**

Work with an EPICS positioning motor (for precise positioning) in [Bluesky](http://blueskyproject.io/bluesky/) and [related tools](http://blueskyproject.io/).

First, we'll connect with an EPICS motor (using [ophyd](http://blueskyproject.io/ophyd/)), and then use the Bluesky software to scan the motor (with the scaler from [basic_scaler](_basic_a.ipynb)).

Load `ophyd` device support for the `EpicsMotor` and connect with one EPICS motor channel.  We have a synApps XXX-style IOC with the prefix `gp:`. It has a scaler, 16 soft channel motors, and some other support we'll ignore in this lesson.

**note**:  This tutorial expects to find an EPICS IOC on the local network configured as a synApps `xxx` [IOC](https://github.com/epics-modules/xxx) with prefix `gp:`.

<details>
A docker container is available to provide this IOC.  See this URL for instructions:  https://github.com/prjemian/epics-docker/tree/main/v1.1/n5_custom_synApps/README.md
</details>

In [1]:
from ophyd import EpicsMotor
m1 = EpicsMotor("gp:m1", name="m1")
m1.wait_for_connection()

Show the current value of the motor (that's the `.RBV` field, in case you were interested).

In [2]:
print(f"{m1.position = }")

m1.position = 1.0


Connect the scaler (as was done in the [basic_scaler](_basic_a.ipynb) lesson).  Define some of the channel names and clear out others.

In [3]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH("gp:scaler1", name="scaler")
scaler.wait_for_connection()

# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")

Use only the channels with EPICS names, those are the *interesting* channels.

In [4]:
scaler.select_channels(None)
scaler.read()

OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1684257260.223683}),
             ('I0', {'value': 7.0, 'timestamp': 1684257260.223683}),
             ('scint', {'value': 8.0, 'timestamp': 1684257260.223683}),
             ('scaler_time', {'value': 1.6, 'timestamp': 1684257260.223683})])

Create a RunEngine but do not connect it with a data collection strategy.  That will come in the next lessons.

In [5]:
from bluesky import RunEngine
import bluesky.plans as bp
RE = RunEngine({})

Run a step scan using the motor and the scaler.

In [6]:
RE(bp.scan([scaler], m1, -1, 1, 5))

('588b3330-2fc0-48db-8943-0383db1d22f0',)

Ah, yes.  Nothing to see here since we did not setup anything to receive the *documents* from the RunEngine.  Here's the basic callback from the [basic_scaler](_basic_a.ipynb) lesson.

In [7]:
import pprint
def myCallback(key, doc):
    print()
    print(key, len(doc))
    pprint.pprint(doc)

Repeat the same scan but handle the document stream with `myCallback()`.

In [8]:
RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)


start 15
{'detectors': ['scaler'],
 'hints': {'dimensions': [(['m1'], 'primary')]},
 'motors': ('m1',),
 'num_intervals': 4,
 'num_points': 5,
 'plan_args': {'args': ["EpicsMotor(prefix='gp:m1', name='m1', "
                        'settle_time=0.0, timeout=None, '
                        "read_attrs=['user_readback', 'user_setpoint'], "
                        "configuration_attrs=['user_offset', "
                        "'user_offset_dir', 'velocity', 'acceleration', "
                        "'motor_egu'])",
                        -1,
                        1],
               'detectors': ["ScalerCH(prefix='gp:scaler1', name='scaler', "
                             "read_attrs=['channels', 'channels.chan01', "
                             "'channels.chan01.s', 'channels.chan02', "
                             "'channels.chan02.s', 'channels.chan03', "
                             "'channels.chan03.s', 'time'], "
                             "configuration_attrs=['channels', "
  

('27c27453-9632-4f8c-b0a7-602c5db0a2e1',)

In [9]:
from ophyd import EpicsMotor
m1 = EpicsMotor("gp:m1", name="m1")
m1.wait_for_connection()

Show the current value of the motor (that's the `.RBV` field, in case you were interested).

In [10]:
print(f"{m1.position = }")

m1.position = 1.0


Connect the scaler (as was done in the [basic_scaler](_basic_a.ipynb) lesson).  Define some of the channel names and clear out others.

In [11]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH("gp:scaler1", name="scaler")
scaler.wait_for_connection()

# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")

Use only the channels with EPICS names, those are the *interesting* channels.

In [12]:
scaler.select_channels(None)
scaler.read()

OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1684257298.403889}),
             ('I0', {'value': 9.0, 'timestamp': 1684257298.403889}),
             ('scint', {'value': 7.0, 'timestamp': 1684257298.403889}),
             ('scaler_time', {'value': 1.6, 'timestamp': 1684257298.403889})])

Create a RunEngine but do not connect it with a data collection strategy.  That will come in the next lessons.

In [13]:
from bluesky import RunEngine
import bluesky.plans as bp
RE = RunEngine({})

Run a step scan using the motor and the scaler.

In [14]:
RE(bp.scan([scaler], m1, -1, 1, 5))

('d5960bf4-fb29-43a1-9783-e7b9f3a926f3',)

Ah, yes.  Nothing to see here since we did not setup anything to receive the *documents* from the RunEngine.  Here's the basic callback from the [`basic_scaler`](_basic_a.ipynb) notebook.

In [15]:
import pprint
def myCallback(key, doc):
    print()
    print(key, len(doc))
    pprint.pprint(doc)

Repeat the same scan but handle the document stream with `myCallback()`.

In [16]:
RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)


start 15
{'detectors': ['scaler'],
 'hints': {'dimensions': [(['m1'], 'primary')]},
 'motors': ('m1',),
 'num_intervals': 4,
 'num_points': 5,
 'plan_args': {'args': ["EpicsMotor(prefix='gp:m1', name='m1', "
                        'settle_time=0.0, timeout=None, '
                        "read_attrs=['user_readback', 'user_setpoint'], "
                        "configuration_attrs=['user_offset', "
                        "'user_offset_dir', 'velocity', 'acceleration', "
                        "'motor_egu'])",
                        -1,
                        1],
               'detectors': ["ScalerCH(prefix='gp:scaler1', name='scaler', "
                             "read_attrs=['channels', 'channels.chan01', "
                             "'channels.chan01.s', 'channels.chan02', "
                             "'channels.chan02.s', 'channels.chan03', "
                             "'channels.chan03.s', 'time'], "
                             "configuration_attrs=['channels', "
  

('f5cee92c-ecf0-4e92-8ba6-075d574638fe',)

## Summary

We'll show this code as a python program:

```
#!/usr/bin/env python

"""Basic : motor"""

from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp


def myCallback(key, doc):
    print()
    print(key, len(doc))
    pprint.pprint(doc)


m1 = EpicsMotor("gp:m1", name="m1")
m1.wait_for_connection()
print(m1.position)

scaler = ScalerCH("gp:scaler1", name="scaler")
scaler.wait_for_connection()


# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")


scaler.match_names()
scaler.select_channels()
print(scaler.read())

RE = RunEngine({})

RE(bp.scan([scaler], m1, -1, 1, 5))

RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)
```